# Pytorch運算基本練習

## 熟悉Tensors
- Tensors類似矩陣(matrices)或陣列(arrays)。
- Tensors可以編碼輸入、輸出或模型參數。
- Tensors跟NumPy’s ndarrays很像，差別是Tensors可以在GPUs或其他硬體加速器上跑。
- 其實Tensors跟NumPy arrays共用底層記憶體，可以直接橋接(請見[Bridge with NumPy][1])
- Tensors也有在自動微分最佳化(接下來在[Autograd][2]我們會講到)

[1]: https://docs.pytorch.org/tutorials/beginner/blitz/tensor_tutorial.html#bridge-to-np-label "Tensors跟NumPy arrays共用底層記憶體" 
[2]: https://docs.pytorch.org/tutorials/beginner/basics/autograd_tutorial.html "自動微分"

In [1]:
import torch
x = torch.rand(3,5) #維度也可以寫成*[3,5]，代表list展開
print(x)

tensor([[0.6552, 0.9418, 0.4032, 0.3965, 0.0483],
        [0.6124, 0.3633, 0.8570, 0.7142, 0.0670],
        [0.8184, 0.4314, 0.7275, 0.5116, 0.4181]])


In [2]:
print(torch.cuda.is_available()) #確認可以移動到GPU上

True


### 建立Tensors
有很多方式:
1. Directly from data
2. From a NumPy array
3. From another tensor
4. By assigning dimensions and filling them with random or constant values

In [3]:
import torch
import numpy as np

# 1 Directly from data
data = [[1,2], [3,4]]
x_data = torch.tensor(data)
print(f"Tensors Directly from data: \n {x_data} \n")
print("---------------")

# 2 From a NumPy array
data_np = np.array([[1,2], [3,4]])
x_np = torch.from_numpy(data_np)
print(f"Tensors From a NumPy array: \n {x_np} \n")
print("---------------")

# 3 From another tensor
print("Tensors From another tensor:")
x_ones = torch.ones_like(x_data) #保留原tensors properties (shape, datatype)，內容全填1
print(f"Ones Tensor: \n {x_ones} \n")
x_rand = torch.rand_like(x_data, dtype=torch.float) #原tensors的datatype改成float
print(f"Random Tensor: \n {x_rand} \n")
print("---------------")

# 4 Assigning dimensions
shape = (2, 3,) #tuple只有一個元素的時候一定要加上逗號
rand_tensor = torch.rand(2,3) #參數可以是int sequence或是傳入tuple, list都可
ones_tensor = torch.ones(shape)
zeros_tensor = torch.zeros(shape)

print("By assigning dimensions and filling them with random or constant values:")
print(f"Random Tensor: \n {rand_tensor} \n")
print(f"Ones Tensor: \n {ones_tensor} \n")
print(f"Zeros Tensor: \n {zeros_tensor}")
print("---------------")

Tensors Directly from data: 
 tensor([[1, 2],
        [3, 4]]) 

---------------
Tensors From a NumPy array: 
 tensor([[1, 2],
        [3, 4]]) 

---------------
Tensors From another tensor:
Ones Tensor: 
 tensor([[1, 1],
        [1, 1]]) 

Random Tensor: 
 tensor([[0.2017, 0.4208],
        [0.6661, 0.9051]]) 

---------------
By assigning dimensions and filling them with random or constant values:
Random Tensor: 
 tensor([[0.2258, 0.8503, 0.1571],
        [0.9746, 0.5821, 0.5371]]) 

Ones Tensor: 
 tensor([[1., 1., 1.],
        [1., 1., 1.]]) 

Zeros Tensor: 
 tensor([[0., 0., 0.],
        [0., 0., 0.]])
---------------


### Attributes of a Tensor
包含shape, datatype, and the device they are stored.

In [4]:
tensor = torch.rand(2,3)
print(f"張量的shape: {tensor.shape}")
print(f"張量的datatype: {tensor.dtype}")
print(f"張量存放的位置: {tensor.device}")

張量的shape: torch.Size([2, 3])
張量的datatype: torch.float32
張量存放的位置: cpu


### 把tensor移動到GPU
tensor提供超過100種運算，包括: 算術、線性代數、矩陣運算（轉置、索引、切片）、取樣[等][1]

[1]: https://pytorch.org/docs/stable/torch.html "torch docs"

In [5]:
if torch.cuda.is_available():
    tensor = tensor.to('cuda') #一定要=，因為tensor.to()是回傳一份新的tensor，不是參照
print(f"張量存放的位置: {tensor.device}")

張量存放的位置: cuda:0


### Numpy-like的索引和切片
`a:b`代表索引a~b-1  
`a:b:c`代表索引a, a+c, a+2c 一直到超過b就停止(不包含b，如果c是正的，最多到b-1；如果c是負的，最多到b+1)  
`:`代表該維度的全部索引  
`...`代表剩餘全部維度全部索引，可以放在開頭、中間、結尾，**但只能出現一次**  
`tensor[0]`如果索引在開頭，可以不寫後面的索引，代表選取剩餘全部維度全部索引

In [6]:
tensor = torch.ones(2,3,4)
print(f"First Batch: \n {tensor[0]} \n") #或是tensor[0,:,:] 或tensor[0,...]
print(f"First Batch -> Second Row: \n {tensor[0,1,:]} \n") #或是tensor[0,1]
print(f"Last Column: \n {tensor[..., -1]} \n") #或是tensor[:,:,-1]
tensor[...,1] = 0
print(f"After assign 0 to second column: \n {tensor} \n")

First Batch: 
 tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]]) 

First Batch -> Second Row: 
 tensor([1., 1., 1., 1.]) 

Last Column: 
 tensor([[1., 1., 1.],
        [1., 1., 1.]]) 

After assign 0 to second column: 
 tensor([[[1., 0., 1., 1.],
         [1., 0., 1., 1.],
         [1., 0., 1., 1.]],

        [[1., 0., 1., 1.],
         [1., 0., 1., 1.],
         [1., 0., 1., 1.]]]) 



### 連接tensors
**torch.cat 是「連接」，torch.stack 是「堆疊」。**

`torch.cat([tensor, tensor, tensor], dim=)`
> torch.cat() 用來「沿著指定的維度，把多個 tensor 串接起來（concatenate）」，前提是要串接的 tensor 在非指定維度上形狀要一致，**不會增加新的維度，但會增加指定維度的元素個數**。這邊傳入`tensor1`, `tensor2`, `tensor3`，3個tensor就會在指定維度上合併所有tensor的元素，假設大家都是2x2，`dim=1`就會變成2x6，`dim=0`就會變成6x2。

`torch.stack([tensor1, tensor2, tensor3], dim=)`
> torch.stack() 用來將**多個 shape 完全一樣的 tensor 沿著一個「新維度」堆疊起來，會增加一個新的維度在指定`dim`**。這邊傳入`tensor1`, `tensor2`, `tensor3`，3個tensor就會在指定維度上新增3個數量，假設大家都是2x2，`dim=1`就會變成2x3x2。

In [7]:
tensor1 = torch.ones(2,2)
tensor2 = torch.full_like(tensor1, 2) #建立都是2的矩陣，維度和tensor1一樣
tensor3 = torch.full_like(tensor1, 3) #建立都是3的矩陣，維度和tensor1一樣

print(f"tensor1:\n{tensor1}")
print(f"tensor2:\n{tensor2}")
print(f"tensor3:\n{tensor3}")

print("===cat===")
tensor_cat = torch.cat([tensor1, tensor2, tensor3], dim=1)
print(f"torch.cat([tensor1, tensor2, tensor3], dim=1):\n{tensor_cat}")
print(f"shape: {tensor_cat.shape}")
print("---")
tensor_cat = torch.cat([tensor1, tensor2, tensor3], dim=0)
print(f"torch.cat([tensor1, tensor2, tensor3], dim=0):\n{tensor_cat}")
print(f"shape: {tensor_cat.shape}")
print("=========\n")

print("===stack===")
tensor_stack = torch.stack([tensor1, tensor2, tensor3], dim=1)
print(f"torch.stack([tensor1, tensor2, tensor3], dim=1):\n{tensor_stack}")
print(f"shape: {tensor_stack.shape}")
print("---")
tensor_stack = torch.stack([tensor1, tensor2, tensor3], dim=0)
print(f"torch.stack([tensor1, tensor2, tensor3], dim=0):\n{tensor_stack}")
print(f"shape: {tensor_stack.shape}")
print("=========")

tensor1:
tensor([[1., 1.],
        [1., 1.]])
tensor2:
tensor([[2., 2.],
        [2., 2.]])
tensor3:
tensor([[3., 3.],
        [3., 3.]])
===cat===
torch.cat([tensor1, tensor2, tensor3], dim=1):
tensor([[1., 1., 2., 2., 3., 3.],
        [1., 1., 2., 2., 3., 3.]])
shape: torch.Size([2, 6])
---
torch.cat([tensor1, tensor2, tensor3], dim=0):
tensor([[1., 1.],
        [1., 1.],
        [2., 2.],
        [2., 2.],
        [3., 3.],
        [3., 3.]])
shape: torch.Size([6, 2])

===stack===
torch.stack([tensor1, tensor2, tensor3], dim=1):
tensor([[[1., 1.],
         [2., 2.],
         [3., 3.]],

        [[1., 1.],
         [2., 2.],
         [3., 3.]]])
shape: torch.Size([2, 3, 2])
---
torch.stack([tensor1, tensor2, tensor3], dim=0):
tensor([[[1., 1.],
         [1., 1.]],

        [[2., 2.],
         [2., 2.]],

        [[3., 3.],
         [3., 3.]]])
shape: torch.Size([3, 2, 2])


### 數學運算(Arithmetic operations)

#### 矩陣乘法(Matrix Multiplication)
A @ B: A的最後一個維度(column)size要跟B倒數第二個維度(row)一樣，前面其他維度也要一樣，例如batch=10。例: A(10,2,3) @ B(10,3,4) = C(10,2,4)
>Tips: 只要看最後兩個維度，前面所有維度當作batch，batch要一樣。

用法: `@` 或 `tensor.matmul()` 或 `torch.matmul()`

In [8]:
tensor1 = torch.full([2,3], 1.0)
tensor2 = torch.full([2,3], 2.0)

print(f"tensor1:\n{tensor1}")
print(f"tensor2:\n{tensor2}")
print(f"tensor2.T:\n{tensor2.T}")

print("===Matrix Multiplication===")
y1 = tensor1 @ tensor2.T
print(f"tensor1 @ tensor2.T:\n{y1}")
y2 = tensor1.matmul(tensor2.T)
print(f"tensor1.matmul(tensor2.T):\n{y2}")
# 先創造一個可以儲存結果的矩陣。
# tensor1.shape是2x3，tensor2.T.shape是3x2，兩個矩陣相乘後會變成2x2，也就是取tensor1.shape[0]xtensor2.T.shape[1]
y3 = torch.rand([tensor1.shape[0], tensor2.T.shape[1]])
torch.matmul(tensor1, tensor2.T, out=y3) #儲存結果到y3
print(f"torch.matmul(tensor1, tensor2.T, out=y3):\n{y3}")
print("===========================")

tensor1:
tensor([[1., 1., 1.],
        [1., 1., 1.]])
tensor2:
tensor([[2., 2., 2.],
        [2., 2., 2.]])
tensor2.T:
tensor([[2., 2.],
        [2., 2.],
        [2., 2.]])
===Matrix Multiplication===
tensor1 @ tensor2.T:
tensor([[6., 6.],
        [6., 6.]])
tensor1.matmul(tensor2.T):
tensor([[6., 6.],
        [6., 6.]])
torch.matmul(tensor1, tensor2.T, out=y3):
tensor([[6., 6.],
        [6., 6.]])


#### 逐元素乘法(Element-wise Multiplication)
shape 必須完全一樣

語法: `*` 或 `tensor.mul()` 或 `torch.mul()`

In [9]:
tensor1 = torch.full([2,2], 2.0)
tensor2 = torch.full([2,2], 3.0)

print(f"tensor1:\n{tensor1}")
print(f"tensor2:\n{tensor2}")

print("===Element-wise Multiplication===")
z1 = tensor1 * tensor2
print(f"tensor1 * tensor2:\n{z1}")
z2 = tensor1.mul(tensor2)
print(f"tensor1.mul(tensor2):\n{z2}")
z3 = torch.rand_like(tensor1) #z3用來儲存結果
torch.mul(tensor1, tensor2, out=z3)
print(f"torch.mul(tensor1, tensor2, out=z3):\n{z3}")
print("======")

tensor1:
tensor([[2., 2.],
        [2., 2.]])
tensor2:
tensor([[3., 3.],
        [3., 3.]])
===Element-wise Multiplication===
tensor1 * tensor2:
tensor([[6., 6.],
        [6., 6.]])
tensor1.mul(tensor2):
tensor([[6., 6.],
        [6., 6.]])
torch.mul(tensor1, tensor2, out=z3):
tensor([[6., 6.],
        [6., 6.]])


### 一個元素的tensor轉型成python的數值型別(float或int)

In [10]:
tensor = torch.ones([2,2])
print(f"tensor:\n{tensor}")
agg = tensor.sum()
print(f"agg before item(): {agg}, type: {type(agg)}, dtype: {agg.dtype}")
agg_item = agg.item()
print(f"agg after item(): {agg_item}, type: {type(agg_item)}")

tensor:
tensor([[1., 1.],
        [1., 1.]])
agg before item(): 4.0, type: <class 'torch.Tensor'>, dtype: torch.float32
agg after item(): 4.0, type: <class 'float'>


### 就地操作(In-place operations)
語法: 物件方法名稱後綴加上`_`或是`torch.函數名稱()`加上參數`out=變數名稱`。  
In-place operation：會直接改變原本的 tensor 值，而不是建立一個新的 tensor。  
> 可節省記憶體，但是不利於自動微分，因為歷史的計算圖會遺失，導致 `.backward()` 會出錯

In [11]:
tensor = torch.zeros([2,2])
print(f"tensor:\n{tensor}\n")
tensor.add(3) #要寫成tensor = tensor.add(3)，因為是回傳新的值，不是inplace
print(f"tensor after .add(3):\n{tensor}\n")
tensor.add_(5) #inplace 物件方法風格
print(f"tensor after .add_(5):\n{tensor}\n")
torch.add(tensor, 5, out=tensor) #inplace 函數風格
print(f"torch.add(tensor, 5, out=tensor):\n{tensor}\n")

tensor:
tensor([[0., 0.],
        [0., 0.]])

tensor after .add(3):
tensor([[0., 0.],
        [0., 0.]])

tensor after .add_(5):
tensor([[5., 5.],
        [5., 5.]])

torch.add(tensor, 5, out=tensor):
tensor([[10., 10.],
        [10., 10.]])



### Bridge with NumPy
在CPU上的Tensors可以跟NumPy arrays共享底層記憶體位置，改變其中一個另一個也會跟著改變。

#### Tensor to NumPy array

In [12]:
t = torch.ones([2,2])
print(f"t:\n{t}")

n = t.numpy()
print(f"n:\n{n}")

t:
tensor([[1., 1.],
        [1., 1.]])
n:
[[1. 1.]
 [1. 1.]]


##### Tensors會改變Numpy arrays

In [13]:
t.add_(2) #要inplace才會改變numpy
print(f"t:\n{t}")
print(f"n:\n{n}")

t:
tensor([[3., 3.],
        [3., 3.]])
n:
[[3. 3.]
 [3. 3.]]


#### NumPy array to Tensor

In [14]:
n = np.ones([2,2])
print(f"n:\n{n}")

t = torch.from_numpy(n)
print(f"t:\n{t}")


n:
[[1. 1.]
 [1. 1.]]
t:
tensor([[1., 1.],
        [1., 1.]], dtype=torch.float64)


##### Numpy arrays會改變Tensors

In [15]:
np.add(n, 2, out=n) #要inplace操作才會改變numpy及tensor的同個記憶體位置裡面的值
print(f"n:\n{n}")
print(f"t:\n{t}")

n:
[[3. 3.]
 [3. 3.]]
t:
tensor([[3., 3.],
        [3., 3.]], dtype=torch.float64)
